# Forward Pricing Problem Set

This notebook covers forward pricing for stocks, commodities, and currencies, including the concept of prepaid forwards.

## 1. Theoretical Foundations

### 1.1 Forward Contracts

A **forward contract** is an agreement to buy or sell an asset at a specified future time $T$ for a price $F$ (the forward price) agreed upon today.

### 1.2 Forward Price Formulas

All formulas below assume **continuous compounding**.

#### 1.2.1 Stock with Continuous Dividend Yield

For a stock with spot price $S$, continuous dividend yield $q$, and risk-free rate $r$:

$$F_0^T = S_0 \cdot e^{(r-q)T}$$

**Intuition**: You earn the risk-free rate $r$ but give up the dividend yield $q$.

#### 1.2.2 Commodity with Lease Rate (Convenience Yield)

For a commodity with spot price $S$, continuous lease rate $\delta$ (or convenience yield), and risk-free rate $r$:

$$F_0^T = S_0 \cdot e^{(r-\delta)T}$$

**Intuition**: Similar to dividends, the lease rate represents the benefit of holding the physical commodity.

#### 1.2.3 Currency (Covered Interest Rate Parity)

For a currency pair where $S$ is the spot exchange rate (domestic per foreign), $r_d$ is the domestic risk-free rate, and $r_f$ is the foreign risk-free rate:

$$F_0^T = S_0 \cdot e^{(r_d - r_f)T}$$

**Important**: The exchange rate convention matters!
- If $S$ = USDCAD (price of 1 USD in CAD), then:
  - Domestic rate = CAD rate ($r_{CAD}$)
  - Foreign rate = USD rate ($r_{USD}$)
  - $F_{USDCAD} = S_{USDCAD} \cdot e^{(r_{CAD} - r_{USD})T}$

- If $S$ = CADUSD (price of 1 CAD in USD), then:
  - Domestic rate = USD rate ($r_{USD}$)
  - Foreign rate = CAD rate ($r_{CAD}$)
  - $F_{CADUSD} = S_{CADUSD} \cdot e^{(r_{USD} - r_{CAD})T}$

### 1.3 Prepaid Forward

A **prepaid forward** is the present value of the forward price. Instead of paying $F$ at time $T$, you pay $F^P$ today.

$$F_0^{P,T} = F_0^T \cdot e^{-rT}$$

Substituting the forward price formulas:

- **Stock**: $F_0^{P,T} = S_0 \cdot e^{-qT}$
- **Commodity**: $F_0^{P,T} = S_0 \cdot e^{-\delta T}$
- **Currency**: $F_0^{P,T} = S_0 \cdot e^{-r_f T}$

**Note**: For currency, the prepaid forward uses the **foreign** risk-free rate.

### 1.4 Key Relationships

#### Reciprocal Property for Currency Forwards

For currency pairs:
$$F_{USDCAD} = \frac{1}{F_{CADUSD}}$$

**However, this does NOT hold for prepaid forwards!**

$$F^P_{USDCAD} \neq \frac{1}{F^P_{CADUSD}}$$

(unless $r_{USD} = r_{CAD}$)

## 2. Problem Set

We will work through the following problems:

**Problem 1**: Calculate forward price for a dividend-paying stock

**Problem 2**: Calculate dividend yield from forward price

**Problem 3**: Calculate forward price for a commodity

**Problem 4**: Calculate lease rate from forward price

**Problem 5**: Calculate currency forward prices (USDCAD and CADUSD)

**Problem 6**: Verify reciprocal relationship for forwards and prepaid forwards

## 3. Setup and Data Generation

In [2]:
import numpy as np
import pandas as pd

# Set random seed for reproducibility
np.random.seed(42)

In [ ]:
# Generate problem data with nice round numbers

# Stock data for Problem 1
S_stock = np.round(np.random.uniform(80, 150), 0)  # Round to nearest dollar
r_stock = np.round(np.random.uniform(0.03, 0.08), 3)  # Round to 0.1%
q_stock = np.round(np.random.uniform(0.01, 0.04), 3)  # Round to 0.1%
T_stock_options = [0.25, 0.5, 0.75, 1.0]  # 3, 6, 9, or 12 months
T_stock = np.random.choice(T_stock_options)

# Commodity data for Problem 3
S_commodity = np.round(np.random.uniform(1000, 2000), 0)  # Round to nearest dollar
r_commodity = np.round(np.random.uniform(0.03, 0.07), 3)  # Round to 0.1%
delta_commodity = np.round(np.random.uniform(0.01, 0.04), 3)  # Round to 0.1%
T_commodity_options = [0.25, 0.5, 0.75, 1.0]
T_commodity = np.random.choice(T_commodity_options)

# Currency data for Problem 5
S_USDCAD = np.round(np.random.uniform(1.20, 1.50), 2)  # Round to 2 decimals
r_CAD = np.round(np.random.uniform(0.03, 0.07), 3)  # Round to 0.1%
r_USD = np.round(np.random.uniform(0.02, 0.06), 3)  # Round to 0.1%
T_fx_options = [0.25, 0.5, 0.75, 1.0]
T_fx = np.random.choice(T_fx_options)

# For Problem 2: Generate forward price from parameters, then ask to find q
S_stock2 = np.round(np.random.uniform(100, 200), 0)
r_stock2 = np.round(np.random.uniform(0.04, 0.08), 3)
q_stock2_actual = np.round(np.random.uniform(0.02, 0.05), 3)
T_stock2_options = [0.5, 0.75, 1.0, 1.5]
T_stock2 = np.random.choice(T_stock2_options)
# Calculate the forward price that corresponds to these parameters
F_stock2 = np.round(S_stock2 * np.exp((r_stock2 - q_stock2_actual) * T_stock2), 2)

# For Problem 4: Generate forward price from parameters, then ask to find delta
S_commodity2 = np.round(np.random.uniform(1500, 2500), 0)
r_commodity2 = np.round(np.random.uniform(0.03, 0.07), 3)
delta_commodity2_actual = np.round(np.random.uniform(0.015, 0.04), 3)
T_commodity2_options = [0.5, 0.75, 1.0]
T_commodity2 = np.random.choice(T_commodity2_options)
# Calculate the forward price that corresponds to these parameters
F_commodity2 = np.round(S_commodity2 * np.exp((r_commodity2 - delta_commodity2_actual) * T_commodity2), 2)

print("="*70)
print("DATA GENERATION SUMMARY")
print("="*70)

print("\nProblem 1 - Stock Forward Price:")
print(f"  S₀ = ${S_stock:.0f}, r = {r_stock*100:.1f}%, q = {q_stock*100:.1f}%, T = {T_stock:.2f} years")

print("\nProblem 2 - Implied Dividend Yield:")
print(f"  S₀ = ${S_stock2:.0f}, F = ${F_stock2:.2f}, r = {r_stock2*100:.1f}%, T = {T_stock2:.2f} years")

print("\nProblem 3 - Commodity Forward Price:")
print(f"  S₀ = ${S_commodity:.0f}, r = {r_commodity*100:.1f}%, δ = {delta_commodity*100:.1f}%, T = {T_commodity:.2f} years")

print("\nProblem 4 - Implied Lease Rate:")
print(f"  S₀ = ${S_commodity2:.0f}, F = ${F_commodity2:.2f}, r = {r_commodity2*100:.1f}%, T = {T_commodity2:.2f} years")
print(f"  (Actual δ used to generate F: {delta_commodity2_actual*100:.1f}%)")

print("\nProblem 5 & 6 - Currency Forwards:")
print(f"  S_USDCAD = {S_USDCAD:.2f}, r_CAD = {r_CAD*100:.1f}%, r_USD = {r_USD*100:.1f}%, T = {T_fx:.2f} years")

print("\n" + "="*70)
print("Data Generated Successfully!")
print("="*70)

## 4. Solutions

### Problem 1: Calculate Forward Price for a Dividend-Paying Stock

**Given:**
- Spot price: $S_0 = \$100$
- Risk-free rate: $r = 5\%$ per annum
- Dividend yield: $q = 2\%$ per annum
- Time to maturity: $T = 1$ year

**Find:** Forward price $F_0^T$

In [4]:
print("="*70)
print("PROBLEM 1: Forward Price for Dividend-Paying Stock")
print("="*70)

print(f"\nGiven:")
print(f"  S₀ = ${S_stock:.2f}")
print(f"  r  = {r_stock*100:.1f}%")
print(f"  q  = {q_stock*100:.1f}%")
print(f"  T  = {T_stock:.1f} year")

print(f"\nFormula: F₀ᵀ = S₀ · e^((r-q)T)")

# Calculate forward price
F_stock = S_stock * np.exp((r_stock - q_stock) * T_stock)

print(f"\nCalculation:")
print(f"  F₀ᵀ = {S_stock:.2f} × e^(({r_stock:.3f} - {q_stock:.3f}) × {T_stock:.1f})")
print(f"      = {S_stock:.2f} × e^({(r_stock - q_stock) * T_stock:.4f})")
print(f"      = {S_stock:.2f} × {np.exp((r_stock - q_stock) * T_stock):.6f}")
print(f"      = ${F_stock:.2f}")

print(f"\n" + "="*70)
print(f"ANSWER: The 1-year forward price is ${F_stock:.2f}")
print("="*70)

PROBLEM 1: Forward Price for Dividend-Paying Stock

Given:
  S₀ = $100.00
  r  = 5.0%
  q  = 2.0%
  T  = 1.0 year

Formula: F₀ᵀ = S₀ · e^((r-q)T)

Calculation:
  F₀ᵀ = 100.00 × e^((0.050 - 0.020) × 1.0)
      = 100.00 × e^(0.0300)
      = 100.00 × 1.030455
      = $103.05

ANSWER: The 1-year forward price is $103.05


### Problem 2: Calculate Dividend Yield from Forward Price

**Given:**
- Spot price: $S_0 = \$150$
- Forward price: $F_0^T = \$156.82$
- Risk-free rate: $r = 6\%$ per annum
- Time to maturity: $T = 1.5$ years

**Find:** Implied dividend yield $q$

In [5]:
print("="*70)
print("PROBLEM 2: Calculate Dividend Yield from Forward Price")
print("="*70)

print(f"\nGiven:")
print(f"  S₀  = ${S_stock2:.2f}")
print(f"  F₀ᵀ = ${F_stock2:.2f}")
print(f"  r   = {r_stock2*100:.1f}%")
print(f"  T   = {T_stock2:.1f} years")

print(f"\nFormula: F₀ᵀ = S₀ · e^((r-q)T)")
print(f"\nSolving for q:")
print(f"  e^((r-q)T) = F₀ᵀ / S₀")
print(f"  (r-q)T = ln(F₀ᵀ / S₀)")
print(f"  r - q = ln(F₀ᵀ / S₀) / T")
print(f"  q = r - ln(F₀ᵀ / S₀) / T")

# Calculate implied dividend yield
q_implied = r_stock2 - np.log(F_stock2 / S_stock2) / T_stock2

print(f"\nCalculation:")
print(f"  F₀ᵀ / S₀ = {F_stock2:.2f} / {S_stock2:.2f} = {F_stock2/S_stock2:.6f}")
print(f"  ln(F₀ᵀ / S₀) = ln({F_stock2/S_stock2:.6f}) = {np.log(F_stock2/S_stock2):.6f}")
print(f"  ln(F₀ᵀ / S₀) / T = {np.log(F_stock2/S_stock2):.6f} / {T_stock2:.1f} = {np.log(F_stock2/S_stock2)/T_stock2:.6f}")
print(f"  q = {r_stock2:.3f} - {np.log(F_stock2/S_stock2)/T_stock2:.6f}")
print(f"    = {q_implied:.6f}")
print(f"    = {q_implied*100:.2f}%")

print(f"\n" + "="*70)
print(f"ANSWER: The implied dividend yield is {q_implied*100:.2f}%")
print("="*70)

# Verify
print(f"\nVerification:")
F_verify = S_stock2 * np.exp((r_stock2 - q_implied) * T_stock2)
print(f"  F₀ᵀ = {S_stock2:.2f} × e^(({r_stock2:.3f} - {q_implied:.6f}) × {T_stock2:.1f})")
print(f"      = ${F_verify:.2f} ✓")

PROBLEM 2: Calculate Dividend Yield from Forward Price

Given:
  S₀  = $150.00
  F₀ᵀ = $156.82
  r   = 6.0%
  T   = 1.5 years

Formula: F₀ᵀ = S₀ · e^((r-q)T)

Solving for q:
  e^((r-q)T) = F₀ᵀ / S₀
  (r-q)T = ln(F₀ᵀ / S₀)
  r - q = ln(F₀ᵀ / S₀) / T
  q = r - ln(F₀ᵀ / S₀) / T

Calculation:
  F₀ᵀ / S₀ = 156.82 / 150.00 = 1.045467
  ln(F₀ᵀ / S₀) = ln(1.045467) = 0.044463
  ln(F₀ᵀ / S₀) / T = 0.044463 / 1.5 = 0.029642
  q = 0.060 - 0.029642
    = 0.030358
    = 3.04%

ANSWER: The implied dividend yield is 3.04%

Verification:
  F₀ᵀ = 150.00 × e^((0.060 - 0.030358) × 1.5)
      = $156.82 ✓


### Problem 3: Calculate Forward Price for a Commodity

**Given:**
- Spot price: $S_0 = \$1500$
- Risk-free rate: $r = 4\%$ per annum
- Lease rate (convenience yield): $\delta = 1.5\%$ per annum
- Time to maturity: $T = 0.5$ years (6 months)

**Find:** Forward price $F_0^T$

In [6]:
print("="*70)
print("PROBLEM 3: Forward Price for a Commodity")
print("="*70)

print(f"\nGiven:")
print(f"  S₀ = ${S_commodity:.2f}")
print(f"  r  = {r_commodity*100:.1f}%")
print(f"  δ  = {delta_commodity*100:.1f}% (lease rate/convenience yield)")
print(f"  T  = {T_commodity:.1f} years ({T_commodity*12:.0f} months)")

print(f"\nFormula: F₀ᵀ = S₀ · e^((r-δ)T)")

# Calculate forward price
F_commodity = S_commodity * np.exp((r_commodity - delta_commodity) * T_commodity)

print(f"\nCalculation:")
print(f"  F₀ᵀ = {S_commodity:.2f} × e^(({r_commodity:.3f} - {delta_commodity:.3f}) × {T_commodity:.1f})")
print(f"      = {S_commodity:.2f} × e^({(r_commodity - delta_commodity) * T_commodity:.5f})")
print(f"      = {S_commodity:.2f} × {np.exp((r_commodity - delta_commodity) * T_commodity):.6f}")
print(f"      = ${F_commodity:.2f}")

print(f"\n" + "="*70)
print(f"ANSWER: The 6-month forward price is ${F_commodity:.2f}")
print("="*70)

PROBLEM 3: Forward Price for a Commodity

Given:
  S₀ = $1500.00
  r  = 4.0%
  δ  = 1.5% (lease rate/convenience yield)
  T  = 0.5 years (6 months)

Formula: F₀ᵀ = S₀ · e^((r-δ)T)

Calculation:
  F₀ᵀ = 1500.00 × e^((0.040 - 0.015) × 0.5)
      = 1500.00 × e^(0.01250)
      = 1500.00 × 1.012578
      = $1518.87

ANSWER: The 6-month forward price is $1518.87


### Problem 4: Calculate Lease Rate from Forward Price

**Given:**
- Spot price: $S_0 = \$2000$
- Forward price: $F_0^T = \$2045.30$
- Risk-free rate: $r = 5\%$ per annum
- Time to maturity: $T = 1$ year

**Find:** Implied lease rate $\delta$

In [7]:
print("="*70)
print("PROBLEM 4: Calculate Lease Rate from Forward Price")
print("="*70)

print(f"\nGiven:")
print(f"  S₀  = ${S_commodity2:.2f}")
print(f"  F₀ᵀ = ${F_commodity2:.2f}")
print(f"  r   = {r_commodity2*100:.1f}%")
print(f"  T   = {T_commodity2:.1f} year")

print(f"\nFormula: F₀ᵀ = S₀ · e^((r-δ)T)")
print(f"\nSolving for δ:")
print(f"  e^((r-δ)T) = F₀ᵀ / S₀")
print(f"  (r-δ)T = ln(F₀ᵀ / S₀)")
print(f"  r - δ = ln(F₀ᵀ / S₀) / T")
print(f"  δ = r - ln(F₀ᵀ / S₀) / T")

# Calculate implied lease rate
delta_implied = r_commodity2 - np.log(F_commodity2 / S_commodity2) / T_commodity2

print(f"\nCalculation:")
print(f"  F₀ᵀ / S₀ = {F_commodity2:.2f} / {S_commodity2:.2f} = {F_commodity2/S_commodity2:.6f}")
print(f"  ln(F₀ᵀ / S₀) = ln({F_commodity2/S_commodity2:.6f}) = {np.log(F_commodity2/S_commodity2):.6f}")
print(f"  ln(F₀ᵀ / S₀) / T = {np.log(F_commodity2/S_commodity2):.6f} / {T_commodity2:.1f} = {np.log(F_commodity2/S_commodity2)/T_commodity2:.6f}")
print(f"  δ = {r_commodity2:.3f} - {np.log(F_commodity2/S_commodity2)/T_commodity2:.6f}")
print(f"    = {delta_implied:.6f}")
print(f"    = {delta_implied*100:.2f}%")

print(f"\n" + "="*70)
print(f"ANSWER: The implied lease rate is {delta_implied*100:.2f}%")
print("="*70)

# Verify
print(f"\nVerification:")
F_verify = S_commodity2 * np.exp((r_commodity2 - delta_implied) * T_commodity2)
print(f"  F₀ᵀ = {S_commodity2:.2f} × e^(({r_commodity2:.3f} - {delta_implied:.6f}) × {T_commodity2:.1f})")
print(f"      = ${F_verify:.2f} ✓")

PROBLEM 4: Calculate Lease Rate from Forward Price

Given:
  S₀  = $2000.00
  F₀ᵀ = $2045.30
  r   = 5.0%
  T   = 1.0 year

Formula: F₀ᵀ = S₀ · e^((r-δ)T)

Solving for δ:
  e^((r-δ)T) = F₀ᵀ / S₀
  (r-δ)T = ln(F₀ᵀ / S₀)
  r - δ = ln(F₀ᵀ / S₀) / T
  δ = r - ln(F₀ᵀ / S₀) / T

Calculation:
  F₀ᵀ / S₀ = 2045.30 / 2000.00 = 1.022650
  ln(F₀ᵀ / S₀) = ln(1.022650) = 0.022397
  ln(F₀ᵀ / S₀) / T = 0.022397 / 1.0 = 0.022397
  δ = 0.050 - 0.022397
    = 0.027603
    = 2.76%

ANSWER: The implied lease rate is 2.76%

Verification:
  F₀ᵀ = 2000.00 × e^((0.050 - 0.027603) × 1.0)
      = $2045.30 ✓


### Problem 5: Calculate Currency Forward Prices (USDCAD and CADUSD)

**Given:**
- Spot USDCAD: $S_{USDCAD} = 1.35$ (1 USD = 1.35 CAD)
- CAD risk-free rate: $r_{CAD} = 4.5\%$ per annum
- USD risk-free rate: $r_{USD} = 3.0\%$ per annum
- Time to maturity: $T = 0.25$ years (3 months)

**Find:** 
- Forward price $F_{USDCAD}$
- Forward price $F_{CADUSD}$

In [8]:
print("="*70)
print("PROBLEM 5: Currency Forward Prices")
print("="*70)

print(f"\nGiven:")
print(f"  S_USDCAD = {S_USDCAD:.4f} (price of 1 USD in CAD)")
print(f"  r_CAD    = {r_CAD*100:.1f}%")
print(f"  r_USD    = {r_USD*100:.1f}%")
print(f"  T        = {T_fx:.2f} years ({T_fx*12:.0f} months)")

print(f"\n" + "-"*70)
print("PART A: USDCAD Forward")
print("-"*70)

print(f"\nFor USDCAD (price of USD in CAD):")
print(f"  - Domestic currency: CAD (use r_CAD)")
print(f"  - Foreign currency: USD (use r_USD)")
print(f"\nFormula: F_USDCAD = S_USDCAD · e^((r_CAD - r_USD)T)")

# Calculate USDCAD forward
F_USDCAD = S_USDCAD * np.exp((r_CAD - r_USD) * T_fx)

print(f"\nCalculation:")
print(f"  F_USDCAD = {S_USDCAD:.4f} × e^(({r_CAD:.3f} - {r_USD:.3f}) × {T_fx:.2f})")
print(f"           = {S_USDCAD:.4f} × e^({(r_CAD - r_USD) * T_fx:.6f})")
print(f"           = {S_USDCAD:.4f} × {np.exp((r_CAD - r_USD) * T_fx):.6f}")
print(f"           = {F_USDCAD:.4f}")

print(f"\n" + "-"*70)
print("PART B: CADUSD Forward")
print("-"*70)

S_CADUSD = 1 / S_USDCAD

print(f"\nFirst, calculate spot CADUSD:")
print(f"  S_CADUSD = 1 / S_USDCAD = 1 / {S_USDCAD:.4f} = {S_CADUSD:.6f}")

print(f"\nFor CADUSD (price of CAD in USD):")
print(f"  - Domestic currency: USD (use r_USD)")
print(f"  - Foreign currency: CAD (use r_CAD)")
print(f"\nFormula: F_CADUSD = S_CADUSD · e^((r_USD - r_CAD)T)")

# Calculate CADUSD forward
F_CADUSD = S_CADUSD * np.exp((r_USD - r_CAD) * T_fx)

print(f"\nCalculation:")
print(f"  F_CADUSD = {S_CADUSD:.6f} × e^(({r_USD:.3f} - {r_CAD:.3f}) × {T_fx:.2f})")
print(f"           = {S_CADUSD:.6f} × e^({(r_USD - r_CAD) * T_fx:.6f})")
print(f"           = {S_CADUSD:.6f} × {np.exp((r_USD - r_CAD) * T_fx):.6f}")
print(f"           = {F_CADUSD:.6f}")

print(f"\n" + "="*70)
print(f"ANSWERS:")
print(f"  F_USDCAD = {F_USDCAD:.4f}")
print(f"  F_CADUSD = {F_CADUSD:.6f}")
print("="*70)

PROBLEM 5: Currency Forward Prices

Given:
  S_USDCAD = 1.3500 (price of 1 USD in CAD)
  r_CAD    = 4.5%
  r_USD    = 3.0%
  T        = 0.25 years (3 months)

----------------------------------------------------------------------
PART A: USDCAD Forward
----------------------------------------------------------------------

For USDCAD (price of USD in CAD):
  - Domestic currency: CAD (use r_CAD)
  - Foreign currency: USD (use r_USD)

Formula: F_USDCAD = S_USDCAD · e^((r_CAD - r_USD)T)

Calculation:
  F_USDCAD = 1.3500 × e^((0.045 - 0.030) × 0.25)
           = 1.3500 × e^(0.003750)
           = 1.3500 × 1.003757
           = 1.3551

----------------------------------------------------------------------
PART B: CADUSD Forward
----------------------------------------------------------------------

First, calculate spot CADUSD:
  S_CADUSD = 1 / S_USDCAD = 1 / 1.3500 = 0.740741

For CADUSD (price of CAD in USD):
  - Domestic currency: USD (use r_USD)
  - Foreign currency: CAD (use r_CAD)

Fo

### Problem 6: Verify Reciprocal Relationship

**Question:** 
1. Is $F_{USDCAD} = \frac{1}{F_{CADUSD}}$?
2. Is $F^P_{USDCAD} = \frac{1}{F^P_{CADUSD}}$?

In [9]:
print("="*70)
print("PROBLEM 6: Reciprocal Relationship for Forwards and Prepaid Forwards")
print("="*70)

print(f"\n" + "-"*70)
print("PART A: Standard Forward Prices")
print("-"*70)

print(f"\nFrom Problem 5:")
print(f"  F_USDCAD = {F_USDCAD:.6f}")
print(f"  F_CADUSD = {F_CADUSD:.6f}")

print(f"\nCheck if F_USDCAD = 1 / F_CADUSD:")
reciprocal_F = 1 / F_CADUSD
print(f"  1 / F_CADUSD = 1 / {F_CADUSD:.6f} = {reciprocal_F:.6f}")

difference_F = abs(F_USDCAD - reciprocal_F)
print(f"\n  F_USDCAD = {F_USDCAD:.6f}")
print(f"  1/F_CADUSD = {reciprocal_F:.6f}")
print(f"  Difference = {difference_F:.10f}")

if difference_F < 1e-6:
    print(f"\n  ✓ YES! F_USDCAD = 1/F_CADUSD (within numerical precision)")
else:
    print(f"\n  ✗ NO! F_USDCAD ≠ 1/F_CADUSD")

print(f"\n" + "-"*70)
print("PART B: Prepaid Forward Prices")
print("-"*70)

print(f"\nCalculate prepaid forward prices:")
print(f"\nFor USDCAD (pay in CAD today, receive USD at maturity):")
print(f"  Formula: F^P_USDCAD = S_USDCAD · e^(-r_USD · T)")
print(f"  (Discount at the USD rate because you're buying USD)")

FP_USDCAD = S_USDCAD * np.exp(-r_USD * T_fx)

print(f"\n  F^P_USDCAD = {S_USDCAD:.4f} × e^(-{r_USD:.3f} × {T_fx:.2f})")
print(f"             = {S_USDCAD:.4f} × e^({-r_USD * T_fx:.6f})")
print(f"             = {S_USDCAD:.4f} × {np.exp(-r_USD * T_fx):.6f}")
print(f"             = {FP_USDCAD:.6f}")

print(f"\nFor CADUSD (pay in USD today, receive CAD at maturity):")
print(f"  Formula: F^P_CADUSD = S_CADUSD · e^(-r_CAD · T)")
print(f"  (Discount at the CAD rate because you're buying CAD)")

FP_CADUSD = S_CADUSD * np.exp(-r_CAD * T_fx)

print(f"\n  F^P_CADUSD = {S_CADUSD:.6f} × e^(-{r_CAD:.3f} × {T_fx:.2f})")
print(f"             = {S_CADUSD:.6f} × e^({-r_CAD * T_fx:.6f})")
print(f"             = {S_CADUSD:.6f} × {np.exp(-r_CAD * T_fx):.6f}")
print(f"             = {FP_CADUSD:.6f}")

print(f"\nCheck if F^P_USDCAD = 1 / F^P_CADUSD:")
reciprocal_FP = 1 / FP_CADUSD
print(f"  1 / F^P_CADUSD = 1 / {FP_CADUSD:.6f} = {reciprocal_FP:.6f}")

difference_FP = abs(FP_USDCAD - reciprocal_FP)
print(f"\n  F^P_USDCAD   = {FP_USDCAD:.6f}")
print(f"  1/F^P_CADUSD = {reciprocal_FP:.6f}")
print(f"  Difference   = {difference_FP:.6f}")

if difference_FP < 1e-6:
    print(f"\n  ✓ YES! F^P_USDCAD = 1/F^P_CADUSD")
else:
    print(f"\n  ✗ NO! F^P_USDCAD ≠ 1/F^P_CADUSD")

print(f"\n" + "="*70)
print("CONCLUSION:")
print("="*70)
print(f"\n1. Standard forwards ARE reciprocals:")
print(f"   F_USDCAD = 1/F_CADUSD ✓")
print(f"\n2. Prepaid forwards are NOT reciprocals:")
print(f"   F^P_USDCAD ≠ 1/F^P_CADUSD ✗")
print(f"\n   This is because prepaid forwards use different discount rates:")
print(f"   - F^P_USDCAD uses r_USD = {r_USD*100:.1f}%")
print(f"   - F^P_CADUSD uses r_CAD = {r_CAD*100:.1f}%")
print(f"\n   The reciprocal relationship only holds if r_USD = r_CAD.")
print("="*70)

PROBLEM 6: Reciprocal Relationship for Forwards and Prepaid Forwards

----------------------------------------------------------------------
PART A: Standard Forward Prices
----------------------------------------------------------------------

From Problem 5:
  F_USDCAD = 1.355072
  F_CADUSD = 0.737968

Check if F_USDCAD = 1 / F_CADUSD:
  1 / F_CADUSD = 1 / 0.737968 = 1.355072

  F_USDCAD = 1.355072
  1/F_CADUSD = 1.355072
  Difference = 0.0000000000

  ✓ YES! F_USDCAD = 1/F_CADUSD (within numerical precision)

----------------------------------------------------------------------
PART B: Prepaid Forward Prices
----------------------------------------------------------------------

Calculate prepaid forward prices:

For USDCAD (pay in CAD today, receive USD at maturity):
  Formula: F^P_USDCAD = S_USDCAD · e^(-r_USD · T)
  (Discount at the USD rate because you're buying USD)

  F^P_USDCAD = 1.3500 × e^(-0.030 × 0.25)
             = 1.3500 × e^(-0.007500)
             = 1.3500 × 0.99252

## 5. Summary Table

In [10]:
# Create comprehensive summary
summary_data = {
    'Problem': [
        'Stock Forward Price',
        'Implied Dividend Yield',
        'Commodity Forward Price',
        'Implied Lease Rate',
        'USDCAD Forward',
        'CADUSD Forward',
        'USDCAD Prepaid Forward',
        'CADUSD Prepaid Forward'
    ],
    'Result': [
        f'${F_stock:.2f}',
        f'{q_implied*100:.2f}%',
        f'${F_commodity:.2f}',
        f'{delta_implied*100:.2f}%',
        f'{F_USDCAD:.4f}',
        f'{F_CADUSD:.6f}',
        f'{FP_USDCAD:.6f}',
        f'{FP_CADUSD:.6f}'
    ]
}

df_summary = pd.DataFrame(summary_data)

print("\n" + "="*70)
print("SUMMARY OF ALL RESULTS")
print("="*70)
print(df_summary.to_string(index=False))

print("\n" + "="*70)
print("KEY INSIGHTS")
print("="*70)
print("\n1. Forward pricing depends on the cost of carry:")
print("   - Stocks: F = S·e^((r-q)T)")
print("   - Commodities: F = S·e^((r-δ)T)")
print("   - Currencies: F = S·e^((r_d-r_f)T)")

print("\n2. Prepaid forwards discount the forward price:")
print("   - F^P = F·e^(-rT)")
print("   - For currencies, use the foreign rate")

print("\n3. Currency forward reciprocal relationship:")
print("   - Standard forwards: F_USDCAD = 1/F_CADUSD ✓")
print("   - Prepaid forwards: F^P_USDCAD ≠ 1/F^P_CADUSD ✗")
print("     (unless r_USD = r_CAD)")
print("="*70)


SUMMARY OF ALL RESULTS
                Problem   Result
    Stock Forward Price  $103.05
 Implied Dividend Yield    3.04%
Commodity Forward Price $1518.87
     Implied Lease Rate    2.76%
         USDCAD Forward   1.3551
         CADUSD Forward 0.737968
 USDCAD Prepaid Forward 1.339913
 CADUSD Prepaid Forward 0.732454

KEY INSIGHTS

1. Forward pricing depends on the cost of carry:
   - Stocks: F = S·e^((r-q)T)
   - Commodities: F = S·e^((r-δ)T)
   - Currencies: F = S·e^((r_d-r_f)T)

2. Prepaid forwards discount the forward price:
   - F^P = F·e^(-rT)
   - For currencies, use the foreign rate

3. Currency forward reciprocal relationship:
   - Standard forwards: F_USDCAD = 1/F_CADUSD ✓
   - Prepaid forwards: F^P_USDCAD ≠ 1/F^P_CADUSD ✗
     (unless r_USD = r_CAD)
